(1) Import 使用的模組進來 , Import library

In [2]:
import pandas as pd
import numpy as np
import csv
import os
import sys
import pickle

In [4]:
#from NameModules.fortune_calculator import moe_data_dict, special_word_dict
from NameModules.fortune_calculator import stroke_total
from NameModules.fortune_calculator import stroke_outside
from NameModules.fortune_calculator import stroke_man
from NameModules.fortune_calculator import stroke_heaven
from NameModules.fortune_calculator import stroke_earth
from NameModules.fortune_calculator import test_name_Fortune_telling
from NameModules.fortune_calculator import get_talent_type
from NameModules.fortune_calculator import get_talent_state
from NameModules.fortune_calculator import get_stroke_state
from NameModules.fortune_calculator import Get_stroke
from NameModules.share_lib import is_chinese, is_number, PrintException
from NameModules.Taiwan_name_seperate import checkLastName, GetLastName, GetFirstName, get_LastName_from_FN, is_biFirstName


'''
checkLastName: 檢查名字的姓氏有無在百家姓的list中
GetLastName: 取姓氏
GetLastName: 取名字
get_LastName_from_FN:
is_biFirstName:是否是疊字名字 
'''
from NameModules.Taiwan_name_seperate import Taiwan_LastName_List, Taiwan_LastName_len1_List, Taiwan_LastName_len2_List
from NameModules.NameModule import contain_Simplified_character , contain_unreadable_character

'''
contain_Simplified_character: find名字是否含非繁體字會用之字的簡體字
contain_unreadable_character: 名字否含不可解讀的編碼的特殊字元
'''

from NameModules.share_lib import is_number,is_chinese, reduce_mem_usage
'''
reduce_mem_usage: 自動將df各欄的型態轉型成可接受的較小size 型態
'''

from gensim.models import word2vec

from hanziconv import HanziConv 


def NamePreprecoess(Name_df):
    ID_col = 'userID'
    if ID_col in Name_df.columns:
        #Drop duplicate name in FB source
        FB_Name_df = Name_df [ Name_df[ID_col].apply(lambda x: type(x)!=float) ]
        ori_len = len(FB_Name_df)
        FB_Name_df = FB_Name_df.drop_duplicates(subset= ID_col ,keep='first',inplace=False)
        print ('Drop duplicate FB name: from ',ori_len,'->',len(FB_Name_df),' drop:',ori_len-len(FB_Name_df))
        
        Name_df = Name_df[ Name_df[ID_col].apply(lambda x: type(x)==float) ]
        Name_df = pd.concat([Name_df, FB_Name_df], ignore_index=True)
    
    #Drop Message is not number
    ori_len = len(Name_df)
    Name_df  = Name_df[ Name_df.message.apply(lambda x:type(x)!=float) ]
    Name_df  = Name_df[ Name_df.message.apply(lambda x: is_number(x)) ]
    print ('Drop Message is not number: ',ori_len,'->',len(Name_df),' drop:',ori_len-len(Name_df))
    ori_len = len(Name_df)
    
    #Drop English name
    ori_len = len(Name_df)
    Name_df = Name_df[ Name_df.name.apply(lambda x: (is_chinese(x))) ]
    print ('Drop English name: from ',ori_len,'->',len(Name_df),' drop:',ori_len - len(Name_df))

    ori_len = len(Name_df)
    #Drop last name is not in Taiwan all last name
    Name_df = Name_df[Name_df.name.apply(lambda x: ( checkLastName(x)))]
    print ('Dron last name is not in Taiwan last name list :',ori_len,'->',len(Name_df),' drop:',ori_len - len(Name_df))
    ori_len = len(Name_df)

    Name_df['LastName'] = Name_df.name.apply(lambda x: (GetLastName(x)))
    Name_df['FirstName'] = Name_df.name.apply(lambda x: (GetFirstName(x)))

    #Drop First name is longer than 3 
    Name_df = Name_df[Name_df.FirstName.apply(lambda x: len(x)<3 )]
    print ('Drop First name is longer than 3  :',ori_len,'->',len(Name_df),' drop:',ori_len - len(Name_df))
    ori_len = len(Name_df)
    
    return Name_df


def Read_Saved_Files():
    pkl_data_path = './pkl_data/'
    '''
    w2v phonetic
    '''
    # 2XX萬名字的涵蓋的中文字 List,共8578種字, 如果把character 變成 編號的時候可以用
    Totalname_list = ''
    with open( pkl_data_path + 'Totalname_list.pkl', 'rb') as handle:
        Totalname_list = pickle.loads(handle.read())

    # 萌典的本體 dict
    moe_data_dict = {}
    with open( pkl_data_path +'moe_data_dict.pkl', 'rb') as handle:
        moe_data_dict = pickle.loads(handle.read())

    # 存同義字或是詞的dict
    common_dict = {}
    with open( pkl_data_path + 'common_dict.pkl', 'rb') as handle:
        common_dict = pickle.loads(handle.read())

    '''
    phonetic
    '''
    son_in_list = []
    with open( pkl_data_path + 'son_in_list.pkl', 'rb') as handle:
        son_in_list = pickle.loads(handle.read())

    mu_in_list = []
    with open( pkl_data_path + 'mu_in_list.pkl', 'rb') as handle:
        mu_in_list = pickle.loads(handle.read())

    special_word_dict = {}
    with open( pkl_data_path + 'special_word_dict.pkl', 'rb') as handle:
        special_word_dict = pickle.loads(handle.read())

    return Totalname_list, moe_data_dict, common_dict, son_in_list, mu_in_list, special_word_dict

def get_x_feature(feature_list , columns):
    f = []
    if 'gender' in feature_list:
        f+=['Male_prob' , 'Female_prob']
    
    if 'uni-gram' in feature_list:
        f+=['FN1','FN2']
        
    for col in columns:
        if 'w2v' in feature_list and '_wv_' in col:
            f+= [col]
        elif 'phonetic' in feature_list and ( 'muin_' in col or 'sonin_' in col):
            f+=[col]
        elif 'fortune_map' in feature_list and ("格_" in col or '三才_' in col):
            f+=[col]
        elif 'Zodiac' in feature_list and 'Zodiac_' in col:
            f+=[col]
        elif 'radical' in feature_list and 'radical_' in col:
            f+=[col]
            
    return f 

def add_word_vector(vector_model,word,n):
    if word in vector_model.wv:
        return vector_model.wv[word][n]
    else:
        if word in common_dict:
            if common_dict[word] in vector_model.wv:
                return vector_model.wv[common_dict[word]][n]
            else:
                return 0
        else:
            return 0
        
def Get_FN_WV( vector_model, FirstName, FN_loc ,WV_loc):
    FN =' '
    if len(FirstName)==2:
        if FN_loc == 1:FN=FirstName[0]
        else:FN=FirstName[1]
    elif FN_loc==2:
        FN=FirstName
    return add_word_vector( vector_model,FN,WV_loc)       

def character_to_index(name,n):
    if n==1:
        if name[0] in Totalname_list:
            return Totalname_list.index(name[0])
        else:
            Totalname_list.append(name[0])
            return Totalname_list.index(name[0])
    if n==2 and len(name)==2:
        if name[1] in Totalname_list:
            return Totalname_list.index(name[1])
        else:
            Totalname_list.append(name[1])
            return Totalname_list.index(name[1])
    return -1

def add_pin_in_column(character, mode):
    # mode 1 = sonin = consonant
    # mode 2 = muin = vowel

    if character == -1:
        return None
    
    #不在字典內的補齊
    specail_word_pinyin_dic = {'艶': 'yàn', '鳯': 'fèng', '恵': 'huì', '姈': 'líng', '寳': 'bǎo', '姫': 'jī', '鑅': 'róng',
                               "玂": "qí", "浤": "hóng", '煊': 'xuān', '斔': 'zhōng', '琜': 'lái', '苰': 'hóng', '玹': 'xuán', '姵': 'pèi', '妏': 'wèn',
                               '妘': 'yún', '珺': 'jùn', '媗': 'xuān', '彣': 'wén', '玹': 'xuán', '瀞': 'jìng', '妡': 'xīn', '琁': 'xuán', '浤': 'hóng', '緁': 'jī',
                               '媜': 'zhēng', '姸': 'yán', '嬅': 'huà', '眞': 'zhēn', '廼': 'nǎi', '寛': 'kuān', '秝': 'lì', '蕥': 'yǎ', '汯': 'hóng', '逹': 'dá', '萓': 'yí',
                               '媃': 'róu', '孋': 'lí', '媁': 'wěi', '祤': 'yǔ', '媄': 'měi', '夆': 'fēng', '蒝': 'yuán', '嬣': 'níng', '砡': 'yù', '芠': 'wén',
                               '姳': 'mǐng', '蔆': 'líng', '菈': 'lā', '鍹': 'xuān', '榳': 'tíng', '錤': 'jī', '憓': 'huì', '潓': 'huì', '瓈': 'lí', '芛': 'wěi',
                               '峮': 'qún', '鋕': 'zhì', '姷': 'yòu', '兪': 'yú', '瑠': 'liú', '嫙': 'xuán', '珅': 'shēn', '暟': 'kǎi', '斈': 'xué', '煐': 'yīng', '淓': 'fāng', '瑨': 'jìn', '嬨': 'cí', '琹': 'qín', '珆': 'yí', '琣': 'pěi',
                               '娪': 'wú', '荺': 'yǔn', '爕': 'xiè', '玶': 'píng', '鋆': 'yún', '愼': 'shèn', '斳': 'qín', '瑈': 'róu', '澪': 'líng', '珦': 'xiàng', '妶': 'xián', '姃': 'zhēng', '薾': 'ěr', '溎': 'guì', '琄': 'xuàn', '琡': 'shū', '瑭': 'táng', '嫆': 'róng'
                               }
    #term = character
    term = Totalname_list[character]
    unkown_dict = {}
    try:
        if term not in moe_data_dict:
            term = HanziConv.toTraditional(term)

        if term == ' ':
            return None

        if term in moe_data_dict:
            for hete in moe_data_dict[term]['heteronyms']:
                if 'pinyin' in hete and moe_data_dict[term]['title'] != '啐':
                    word_p = hete['pinyin']

                    if '（' in (hete['pinyin']):
                        if '（讀音）' in hete['pinyin']:
                            word_p = hete['pinyin'].replace('（讀音）', '')
                        if '（語音）' in hete['pinyin']:
                            word_p = hete['pinyin'].replace('（語音）', '')
                        if '(' in word_p:
                            print(word_p+"!!")

                    for mu in mu_in_list:
                        if mu in word_p:
                            if mode == 'sonin':
                                return word_p[: word_p.index(mu)]
                            else:
                                return mu
            # 找不到字音，看是否是哪個字的異體字
            for hete in moe_data_dict[term]['heteronyms']:
                for define in hete['definitions']:
                    if '異體字' in define['def']:
                        d = define['def']
                        alt_term = d[d.index('「')+1: d.index('」')]
                        # print(alt_term,term)

                        for hete2 in moe_data_dict[alt_term]['heteronyms']:
                            if 'pinyin' in hete2 and moe_data_dict[alt_term]['title'] != '啐':

                                word_p = hete2['pinyin']
                                if '（' in (hete2['pinyin']):
                                    if '（讀音）' in hete2['pinyin']:
                                        word_p = hete2['pinyin'].replace(
                                            '（讀音）', '')
                                    if '（語音）' in hete2['pinyin']:
                                        word_p = hete2['pinyin'].replace(
                                            '（語音）', '')
                                    if '(' in word_p:
                                        print(word_p+"!!")

                                for mu in mu_in_list:
                                    if mu in word_p:
                                        if mode == 'sonin':
                                            return word_p[: word_p.index(mu)]
                                        else:
                                            return mu
            # print('在字典內但沒有拼音：',term)
        else:
            # print('不在字典內：',term)

            if term in special_word_dict:
                word_p = special_word_dict[term]['pinyin']
                for mu in mu_in_list:
                    if mu in word_p:
                        if mode == 'sonin':
                            return word_p[: word_p.index(mu)]
                        else:
                            return mu
            else:
                print('拼音不明：',term)
                if term not in unkown_dict:
                    unkown_dict[term] = 1
                else:
                    unkown_dict[term] += 1

#             if len( moe_df[moe_df.字詞名.apply(lambda x: x==term)])>0:
#                 print('不在字典內：',term)
        return unkown_dict
    except Exception as e:
        print(e)
        PrintException()

def add_W2V_feature( Name_df,w2v_Vector_number):
    ###############################################################
    #Add w2v
    #用17G的維基百科+幾十M的萌典範文訓練的w2v, 每個entry是一個字
    moe_model = word2vec.Word2Vec.load("./w2v_data/wiki_moe_100_model.bin")
    
    for i in range(0,w2v_Vector_number):
        Name_df['FN1_wv_'+str(i)]  = Name_df["FirstName"].apply(lambda x:  Get_FN_WV( moe_model , x, 1 , i))
        Name_df['FN2_wv_'+str(i)]  = Name_df["FirstName"].apply(lambda x:  Get_FN_WV( moe_model , x, 2 , i))
        
    Name_df , NA_list = reduce_mem_usage( Name_df )
    
    w2v_feature = get_x_feature( ['w2v'], Name_df.columns)
    print("w2v_feature len",len(w2v_feature))
    ################################################################ 
    return Name_df , w2v_feature

def add_phonetic_feature(Name_df):
    Name_df['FN1'] = Name_df.FirstName.apply(lambda x: character_to_index(x,1))
    Name_df['FN2'] = Name_df.FirstName.apply(lambda x: character_to_index(x,2))
    
    Name_df['FN1_sonin'] = Name_df.FN1.apply(lambda x:  add_pin_in_column(x,'sonin') )
    Name_df['FN1_muin'] = Name_df.FN1.apply(lambda x:  add_pin_in_column(x,'muin') )

    Name_df['FN2_sonin'] = Name_df.FN2.apply(lambda x:  add_pin_in_column(x,'sonin') )
    Name_df['FN2_muin'] = Name_df.FN2.apply(lambda x:  add_pin_in_column(x,'muin') )

    Name_df = pd.get_dummies(Name_df, columns=["FN1_sonin"])
    Name_df = pd.get_dummies(Name_df, columns=["FN1_muin"])
    Name_df = pd.get_dummies(Name_df, columns=["FN2_sonin"])
    Name_df = pd.get_dummies(Name_df, columns=["FN2_muin"])
    
    phonetic_feature = get_x_feature( ['phonetic'] , Name_df.columns)
    print('phonetic_feature len:',len(phonetic_feature))
    
    return Name_df , phonetic_feature

def add_fortune_map_feature(sampled_df):
    sampled_df['天格'] = sampled_df['LastName'].apply(lambda x: get_stroke_state(stroke_heaven (x)) )
    sampled_df['地格'] = sampled_df['LastName'].apply(lambda x: get_stroke_state(stroke_earth (x)) )
    sampled_df['人格'] = sampled_df.apply(lambda x: get_stroke_state(stroke_man (x)) ,axis = 1)
    sampled_df['外格'] = sampled_df.apply(lambda x: get_stroke_state(stroke_outside (x)) ,axis = 1)
    sampled_df['總格'] = sampled_df.apply(lambda x: get_stroke_state(stroke_total (x)) ,axis = 1)
    sampled_df['三才'] = sampled_df.apply(lambda x: get_talent_state(get_talent_type(stroke_heaven(
        x['LastName'])) + get_talent_type(stroke_earth(x['LastName'])) + get_talent_type(stroke_man(x))), axis=1)
    
    sampled_df = pd.get_dummies(sampled_df, columns=["天格"])
    sampled_df = pd.get_dummies(sampled_df, columns=["地格"])
    sampled_df = pd.get_dummies(sampled_df, columns=["人格"])
    sampled_df = pd.get_dummies(sampled_df, columns=["外格"])
    sampled_df = pd.get_dummies(sampled_df, columns=["總格"])
    sampled_df = pd.get_dummies(sampled_df, columns=["三才"])

    fortune_map_feature_list = get_x_feature( ['fortune_map'] , sampled_df.columns)
    print("len on fortune_map_feature_list:",len(fortune_map_feature_list))
    return sampled_df , fortune_map_feature_list

def add_radical_column_age(character):
    if character in moe_data_dict:
        return moe_data_dict[character]['radical'] 
    elif character in special_word_dict :
        return special_word_dict[character]['radical']  
    else:
        return '不明'
        #return -1

def add_radical_feature(sampled_df):
    #FN1存成index
    sampled_df['FN1_radical'] = sampled_df['FN1'].apply(lambda x: add_radical_column_age( Totalname_list[x]))
    sampled_df['FN2_radical'] = sampled_df['FN2'].apply(lambda x: add_radical_column_age( Totalname_list[x]))
    sampled_df = pd.get_dummies(sampled_df, columns=["FN1_radical"])
    sampled_df = pd.get_dummies(sampled_df, columns=["FN2_radical"])

    Radical_feature_list = get_x_feature( ['radical'] , sampled_df.columns)
    print("len of Radical_feature_list: ",len(Radical_feature_list))
    return sampled_df , Radical_feature_list
    
def get_zodiac_from_birthyear(birthyear):
    zodiac_list = ['鼠', '牛', '虎', '兔', '龍', '蛇', '馬', '羊', '猴', '雞', '狗', '豬']
    return zodiac_list[divmod((birthyear - 4), 12)[1]]


def add_zodiac_feature(Name_df):
    Name_df['Zodiac'] = Name_df['message'].apply(lambda x: get_zodiac_from_birthyear(x))
    Name_df = pd.get_dummies(Name_df, columns=["Zodiac"])
    
    Zodiac_feature_list = get_x_feature("Zodiac" , Name_df )
    print("len of Zodiac_feature_list: ",len(Zodiac_feature_list))
    return Name_df , Zodiac_feature_list 

(2)讀檔 Read Files

In [124]:
Path = './NameData/gcname_df_featured.csv'
Name_df = pd.read_csv(Path, dtype='str')
#Name_df [['name', 'BirthYear','FirstName','LastName','gender','message','userID','WritingNumber']].to_csv("./NameData/Total_Name.csv")

In [6]:
Name_df.columns

Index(['name', 'BirthYear', 'FirstName', 'LastName', 'gender', 'message',
       'userID', 'FN1_wv_0', 'FN2_wv_0', 'FN1_wv_1',
       ...
       'Zodiac_狗', 'Zodiac_猴', 'Zodiac_羊', 'Zodiac_虎', 'Zodiac_蛇', 'Zodiac_豬',
       'Zodiac_雞', 'Zodiac_馬', 'Zodiac_鼠', 'Zodiac_龍'],
      dtype='object', length=977)

In [4]:
#Name_df = Name_df[[ 'name', 'BirthYear', 'FirstName', 'LastName', 'gender',
#       'message', 'userID']]

In [125]:
#將object type 復原回int
Name_df['BirthYear'] = Name_df.BirthYear.apply(lambda x: int(x))
Name_df['gender'] = Name_df.gender.apply(lambda x: int(float(x)))
Name_df['message'] =  Name_df.message.apply(lambda x: int(x))

type_dic = {}
for w2v_col in get_x_feature(['w2v'],Name_df):
    type_dic[ w2v_col ] = 'float32'

for col in get_x_feature(['phonetic','fortune_map','Zodiac','radical'],Name_df):
    type_dic[ col ] = 'int8'

In [126]:
Name_df = Name_df.astype( type_dic )

In [9]:
Name_df.dtypes

name         object
BirthYear     int64
FirstName    object
LastName     object
gender        int64
              ...  
Zodiac_豬       int8
Zodiac_雞       int8
Zodiac_馬       int8
Zodiac_鼠       int8
Zodiac_龍       int8
Length: 977, dtype: object

In [ ]:
from collections import Counter
#FN_counter = Counter(Name_df['FirstName'].tolist())

(3) Preparing for learing

(3-1)Import learning fucntion

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [11]:
## from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


def dataset_statistics(dataset):
    """
    Basic statistics of the dataset
    :param dataset: Pandas dataframe
    :return: None, print the basic statistics of the dataset
    """
    print(dataset.describe())

# http://dataaspirant.com/2017/06/26/random-forest-classifier-python-scikit-learn/
# Split train and test


def split_dataset(dataset, train_percentage, feature_headers, target_header):
    """
    Split the dataset with train_percentage
    :param dataset:
    :param train_percentage:
    :param feature_headers:
    :param target_header:
    :return: train_x, test_x, train_y, test_y
    """

    # Split dataset into train and test dataset
    train_x, test_x, train_y, test_y = train_test_split(dataset[feature_headers], dataset[target_header],
                                                        train_size=train_percentage)
    return train_x, test_x, train_y, test_y


def random_forest_classifier(features, target, estimators_num=32,  min_samples_leaf_num=1):
    """
    To train the random forest classifier with features and target data
    :param features:
    :param target:
    :return: trained random forest classifier
    """
    clf = RandomForestClassifier(
        n_estimators= estimators_num, n_jobs=4, min_samples_leaf= min_samples_leaf_num , random_state=42)
    print('estimators_num = ' , estimators_num , 'min_samples_leaf_num = ' , min_samples_leaf_num ,"Training Data len = ", len(features))
    clf.fit(features, target)
    return clf


def random_forest_Regressor(features, target):
    """
    To train the random forest classifier with features and target data
    :param features:
    :param target:
    :return: trained random forest classifier
    """
    clf = RandomForestRegressor(n_estimators=32, n_jobs=4, min_samples_leaf=1)
    clf.fit(features, target)
    return clf


(3-1) Train gender RFC

In [12]:
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn import metrics

In [54]:
Name_df[ ["gender","FirstName"]].describe()

,gender
count,138501.00000
mean,0.58640
std,0.49248
min,0.00000
25%,0.00000
50%,1.00000
75%,1.00000
max,1.00000


In [68]:
y_feature = ['gender']

In [ ]:
feature_combinations = []

(3-2)Make Feature Combinations

In [140]:
feature_list = ['w2v', 'phonetic','fortune_map','Zodiac','radical']

In [13]:
feature_list_gender = ['w2v', 'phonetic','fortune_map','Zodiac','radical','uni-gram']

In [16]:
from itertools import chain, combinations

def get_all_combinations(feature_list):
    feature_combinations = [ x for x in feature_list]
    for i in range(2 , len(feature_list)+1):
        for combo in ( combinations(feature_list , r= i) ):
            temp =[]
            for f in combo:
                temp+=[f]
            feature_combinations += [temp]
    return feature_combinations

In [17]:
feature_combinations_gender = get_all_combinations(feature_list_gender)

In [18]:
x_feature = get_x_feature (['w2v'] , Name_df.columns)

In [19]:
Name_df.FN1

0          256
1         3763
2          884
3          987
4          755
          ... 
138496     203
138497      29
138498    1158
138499      78
138500     249
Name: FN1, Length: 138501, dtype: object

In [20]:
Name_df

,name,BirthYear,FirstName,LastName,gender,message,userID,FN1_wv_0,FN2_wv_0,FN1_wv_1,...,Zodiac_狗,Zodiac_猴,Zodiac_羊,Zodiac_虎,Zodiac_蛇,Zodiac_豬,Zodiac_雞,Zodiac_馬,Zodiac_鼠,Zodiac_龍
0,丁承先,0,承先,丁,1,1940,NaN,-0.082778,-5.796417,4.782166,...,0,0,0,0,0,0,0,0,0,1
1,丁昞原,0,昞原,丁,1,1940,NaN,-1.442101,0.381964,1.215422,...,0,0,0,0,0,0,0,0,0,1
2,方超,0,超,方,1,1940,NaN,0.000000,-3.596860,0.000000,...,0,0,0,0,0,0,0,0,0,1
3,方九龍,0,九龍,方,1,1940,NaN,3.789320,-2.980340,-1.893114,...,0,0,0,0,0,0,0,0,0,1
4,方大錚,0,大錚,方,1,1940,NaN,3.813373,-4.719679,-0.937114,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138496,程麗庭,8,麗庭,程,0,1982,259328941245892,3.107082,1.373621,0.181308,...,1,0,0,0,0,0,0,0,0,0
138497,許家榛,7,家榛,許,0,1979,505551569779505,-0.024527,-0.138811,1.429394,...,0,0,1,0,0,0,0,0,0,0
138498,田崑成,7,崑成,田,1,1978,138985933375787,4.559606,6.068225,2.725899,...,0,0,0,0,0,0,0,1,0,0
138499,筱欣,11,欣,筱,0,1998,1282157788580648,0.000000,-2.398280,0.000000,...,0,0,0,1,0,0,0,0,0,0


(3-3)Add most gender as y feature

In [127]:
sampled_df = Name_df

In [ ]:
#用來扣掉同名字但有不同性別存在的名字
#name_gender_dict內大於0的就是男生,反之女生,0則男女各半
name_gender_dict = {}
def name_gender_count(row):
    name = row['FirstName']
    gender = row['gender']
    if name in name_gender_dict:
        if gender==1:
            name_gender_dict[name]+=1
        else:
            name_gender_dict[name]-=1
    else:
        if gender==1:
            name_gender_dict[name]=1
        else:
            name_gender_dict[name]=-1        
    
sampled_df.apply(lambda row: name_gender_count(row),axis=1)
print(name_gender_dict)

In [26]:
#中性名字沒分第三類別,直接任塞一性別
import random
for name in name_gender_dict:
    if name_gender_dict[name] ==0:
        if random.randint(0,1) == 0:
            name_gender_dict[name] = -1
        else:
            name_gender_dict[name] = 1

In [130]:
def add_most_gender(row):
    name = row['FirstName']
    gender = row['gender']
    
    if name_gender_dict[name] > 0:
        return 1
    if name_gender_dict[name] < 0:
        return 0

    if name_gender_dict[name] == 0:
        return -1
sampled_df['mgender'] = sampled_df.apply(lambda row: add_most_gender(row),axis=1)


In [193]:
sampled_df.gender.value_counts()

1    81217
0    57284
Name: gender, dtype: int64

In [28]:
sampled_df.mgender.value_counts()

1    81475
0    57026
Name: mgender, dtype: int64

(3-4)W2V 做normalize

In [32]:
def _normalize(X, train = True, specified_column = None, X_mean = None, X_std = None):
    # This function normalizes specific columns of X.
    # The mean and standard variance of training data will be reused when processing testing data.
    #
    # Arguments:
    #     X: data to be processed
    #     train: 'True' when processing training data, 'False' for testing data
    #     specific_column: indexes of the columns that will be normalized. If 'None', all columns
    #         will be normalized.
    #     X_mean: mean value of training data, used when train = 'False'
    #     X_std: standard deviation of training data, used when train = 'False'
    # Outputs:
    #     X: normalized data
    #     X_mean: computed mean value of training data
    #     X_std: computed standard deviation of training data

    if specified_column == None:
        #np.arange(X.shape[1])只拿的到pd的shape  [0~n],不能拿到pd的欄位名稱
        specified_column = np.arange(X.shape[1])
        #specified_column = X.columns
    if train:
        #X_mean為一個list放每行的mean
        X_mean = np.mean(X[:, specified_column] ,0).reshape(1, -1)
        X_std  = np.std(X[:, specified_column], 0).reshape(1, -1)

    X[:,specified_column] = (X[:, specified_column] - X_mean) / (X_std + 1e-8)
     
    return X, X_mean, X_std

In [49]:
def w2v_normalize(sampled_df):
    x_feature = get_x_feature ( ['w2v'] , Name_df.columns)
    w2v_np = sampled_df[ x_feature ].to_numpy()
    # Normalize training and testing data
    w2v_np, X_mean, X_std = _normalize(w2v_np, train = True)
    
    idx = 0
    for i in range(100):
        for j in range(1,3):
            col = ("FN{}_wv_{}".format(j,i))
            print(col)
            FN_w2vX =[] 
            for vector_list in (w2v_np):
                FN_w2vX+=[ vector_list[idx]  ]
            idx+=1
            sampled_df[col] = FN_w2vX
    return sampled_df

In [232]:
#未normalized
sampled_df[ x_feature ].describe()

,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,FN2_wv_2,FN1_wv_3,FN2_wv_3,FN1_wv_4,FN2_wv_4,...,FN1_wv_95,FN2_wv_95,FN1_wv_96,FN2_wv_96,FN1_wv_97,FN2_wv_97,FN1_wv_98,FN2_wv_98,FN1_wv_99,FN2_wv_99
count,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,...,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000,138501.000000
mean,-1.149561,-1.142548,0.335170,0.984800,-1.303121,-1.285600,0.969382,1.649082,0.932246,0.639874,...,0.852125,0.855012,-0.818223,-1.027274,0.639393,0.495382,0.975133,0.860273,-1.179080,-1.059375
std,3.492488,3.323843,3.474877,3.337498,3.092279,3.200081,3.298764,3.259781,3.352327,3.217403,...,3.348929,3.137315,3.328003,3.158895,3.503989,3.088225,2.894282,2.827303,3.368849,3.369553
min,-14.552404,-14.552404,-15.155359,-15.155359,-10.770514,-12.120364,-13.059869,-13.059869,-19.813513,-19.813513,...,-13.671687,-15.675972,-11.840435,-13.274806,-10.960847,-10.634512,-13.096807,-13.096807,-11.716203,-11.936399
25%,-3.428356,-3.267654,-1.943416,-1.286454,-3.024722,-3.490335,-0.700949,-0.314655,-0.800308,-1.321648,...,-0.926266,-0.984259,-3.091983,-3.060984,-1.220916,-1.220916,-0.894890,-1.018355,-3.538004,-3.236742
50%,-0.849926,-1.173061,0.181308,1.185356,-1.684384,-1.669534,1.110161,1.810379,0.969153,0.752990,...,0.866414,1.216405,-0.612317,-1.291795,0.270797,0.459639,0.671934,0.629546,-0.906662,-1.021606
75%,0.880691,0.837845,2.884024,3.441082,0.328431,0.566860,3.235934,3.917055,3.014434,2.652030,...,2.981780,2.909477,1.220726,0.991717,2.560374,2.222106,2.960142,2.960142,0.654513,0.882206
max,13.809453,13.809453,13.278703,12.627606,12.577226,13.612901,15.600971,15.600971,12.603272,12.603272,...,13.278911,13.278911,17.067118,17.067118,15.014699,14.567921,13.154280,13.154280,14.519915,14.381725


In [41]:
sampled_df[x_feature]

,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,FN2_wv_2,FN1_wv_3,FN2_wv_3,FN1_wv_4,FN2_wv_4,...,FN1_wv_95,FN2_wv_95,FN1_wv_96,FN2_wv_96,FN1_wv_97,FN2_wv_97,FN1_wv_98,FN2_wv_98,FN1_wv_99,FN2_wv_99
0,-0.082778,-5.796417,4.782166,-0.217218,-3.571659,-5.335327,-8.585439,0.896868,6.623681,0.994031,...,0.497893,2.754097,-6.471545,0.587246,-5.607302,-2.159950,4.919923,0.551634,-4.087385,-2.308148
1,-1.442101,0.381964,1.215422,2.645488,-1.004735,-1.196166,0.030470,4.425450,0.634333,-0.119361,...,1.299436,-0.530058,0.151594,1.345441,-1.183616,0.623379,-0.605316,-0.215416,0.720008,5.855138
2,0.000000,-3.596860,0.000000,-5.960489,0.000000,-0.759137,0.000000,-0.892981,0.000000,-1.062888,...,0.000000,1.683001,0.000000,4.143760,0.000000,4.756191,0.000000,-3.617392,0.000000,-7.906404
3,3.789320,-2.980340,-1.893114,0.727061,8.631498,2.532944,-1.192231,8.816960,0.861374,1.124468,...,3.327790,0.101130,1.703527,6.744654,-3.576160,2.269994,4.723351,4.592242,2.390387,-3.591447
4,3.813373,-4.719679,-0.937114,2.741030,-2.302430,-1.445924,1.900387,-2.508925,4.940345,0.752990,...,-0.926266,2.260610,5.119319,-0.142316,3.783037,-2.275315,1.980577,-1.235366,-2.943175,0.554239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138496,3.107082,1.373621,0.181308,7.103828,-1.789464,-1.372236,1.171252,-0.022872,1.328304,-3.410328,...,0.086999,6.691445,-0.047737,2.491465,-1.211868,-0.915223,1.569784,-1.038645,-4.228339,3.407803
138497,-0.024527,-0.138811,1.429394,3.504970,-5.721419,-0.308425,-1.453798,1.116156,4.826768,0.447931,...,6.056461,-1.345802,4.155251,1.048379,6.082980,-0.555668,-3.041093,1.900405,1.069293,0.627254
138498,4.559606,6.068225,2.725899,-2.063284,-2.807246,-0.077769,4.147959,1.140175,1.654552,-4.686434,...,-1.934218,-7.699350,4.781729,-6.722366,3.168396,2.253842,3.530610,-1.759062,-1.181018,2.278829
138499,0.000000,-2.398280,0.000000,1.651978,0.000000,-1.263631,0.000000,3.889518,0.000000,-1.008286,...,0.000000,4.664589,0.000000,-2.361263,0.000000,5.832688,0.000000,5.729279,0.000000,-0.815454


In [ ]:
sampled_df = w2v_normalize(sampled_df)

In [53]:
sampled_df[x_feature]

,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,FN2_wv_2,FN1_wv_3,FN2_wv_3,FN1_wv_4,FN2_wv_4,...,FN1_wv_95,FN2_wv_95,FN1_wv_96,FN2_wv_96,FN1_wv_97,FN2_wv_97,FN1_wv_98,FN2_wv_98,FN1_wv_99,FN2_wv_99
0,0.305452,-1.400152,1.279761,-0.360157,-0.733616,-1.265512,-2.896495,-0.230757,1.697763,0.110076,...,-0.105775,0.605324,-1.698719,0.511105,-1.782745,-0.859828,1.362965,-0.109164,-0.863296,-0.370606
1,-0.083763,0.458661,0.253320,0.497587,0.096494,0.027948,-0.284626,0.851707,-0.088868,-0.235978,...,0.133569,-0.441484,0.291412,0.751125,-0.520268,0.041447,-0.546061,-0.380466,0.563722,2.052064
2,0.329154,-0.738399,-0.096455,-2.080995,0.421413,0.164516,-0.293863,-0.779829,-0.278090,-0.529237,...,-0.254448,0.263917,0.245861,1.636982,-0.182476,1.379700,-0.336918,-1.583729,0.349996,-2.032036
3,1.414149,-0.552914,-0.641258,-0.077225,3.212729,1.193269,-0.655282,2.198891,-0.021141,0.150617,...,0.739243,-0.240296,0.757739,2.460340,-1.203077,0.574640,1.295047,1.319980,1.059555,-0.751459
4,1.421036,-1.076207,-0.366139,0.526214,-0.323164,-0.050100,0.282229,-1.275553,1.195621,0.035158,...,-0.531035,0.448027,1.784122,0.280149,0.897165,-0.897184,0.347391,-0.741218,-0.523651,0.478883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138496,1.218804,0.757008,-0.044278,1.833425,-0.157277,-0.027073,0.061196,-0.512906,0.118145,-1.258847,...,-0.228470,1.860334,0.231517,1.113919,-0.528331,-0.456770,0.205458,-0.671639,-0.905137,1.325753
138497,0.322131,0.301982,0.314897,0.755110,-1.428821,0.305360,-0.734575,-0.163486,1.161741,-0.059658,...,1.554035,-0.701499,1.494437,0.657084,1.553546,-0.340342,-1.387646,0.367890,0.667403,0.500552
138498,1.634705,2.169416,0.688006,-0.913288,-0.486415,0.377439,0.963569,-0.156118,0.215465,-1.655474,...,-0.832013,-2.726661,1.682683,-1.802881,0.721753,0.569410,0.882943,-0.926446,-0.000575,0.990700
138499,0.329154,-0.377797,-0.096455,0.199904,0.421413,0.006865,-0.293863,0.687299,-0.278090,-0.512266,...,-0.254448,1.214284,0.245861,-0.422298,-0.182476,1.728283,-0.336918,1.722144,0.349996,0.072390


In [58]:
pd.options.display.float_format = '{:,.4f}'.format

In [59]:
#已normalized
sampled_df[ x_feature ].describe()

,FN1_wv_0,FN2_wv_0,FN1_wv_1,FN2_wv_1,FN1_wv_2,FN2_wv_2,FN1_wv_3,FN2_wv_3,FN1_wv_4,FN2_wv_4,...,FN1_wv_95,FN2_wv_95,FN1_wv_96,FN2_wv_96,FN1_wv_97,FN2_wv_97,FN1_wv_98,FN2_wv_98,FN1_wv_99,FN2_wv_99
count,"138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000",...,"138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000","138,501.0000"
mean,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000,0.0000,-0.0000,0.0000,...,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,-0.0000,0.0000,0.0000,-0.0000
std,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
min,-3.8376,-4.0345,-4.4579,-4.8360,-3.0616,-3.3858,-4.2529,-4.5123,-6.1885,-6.3571,...,-4.3369,-5.2692,-3.3120,-3.8772,-3.3106,-3.6040,-4.8620,-4.9366,-3.1278,-3.2280
25%,-0.6525,-0.6394,-0.6557,-0.6805,-0.5567,-0.6890,-0.5064,-0.6024,-0.5168,-0.6097,...,-0.5310,-0.5863,-0.6832,-0.6438,-0.5309,-0.5558,-0.6461,-0.6645,-0.7002,-0.6462
50%,0.0858,-0.0092,-0.0443,0.0601,-0.1233,-0.1200,0.0427,0.0495,0.0110,0.0352,...,0.0043,0.1152,0.0619,-0.0837,-0.1052,-0.0116,-0.1048,-0.0816,0.0809,0.0112
75%,0.5813,0.5958,0.7335,0.7360,0.5276,0.5789,0.6871,0.6957,0.6211,0.6254,...,0.6359,0.6549,0.6127,0.6391,0.5482,0.5591,0.6858,0.7427,0.5443,0.5762
max,4.2832,4.4984,3.7249,3.4885,4.4887,4.6557,4.4355,4.2800,3.4815,3.7184,...,3.7107,3.9601,5.3742,5.7281,4.1026,4.5569,4.2080,4.3483,4.6601,4.5826


In [104]:
sampled_df.head()

,name,BirthYear,FirstName,LastName,gender,message,userID,FN1_wv_0,FN2_wv_0,FN1_wv_1,...,Zodiac_猴,Zodiac_羊,Zodiac_虎,Zodiac_蛇,Zodiac_豬,Zodiac_雞,Zodiac_馬,Zodiac_鼠,Zodiac_龍,mgender
0,丁承先,0,承先,丁,1,1940,NaN,0.3055,-1.4002,1.2798,...,0,0,0,0,0,0,0,0,1,1
1,丁昞原,0,昞原,丁,1,1940,NaN,-0.0838,0.4587,0.2533,...,0,0,0,0,0,0,0,0,1,1
2,方超,0,超,方,1,1940,NaN,0.3292,-0.7384,-0.0965,...,0,0,0,0,0,0,0,0,1,1
3,方九龍,0,九龍,方,1,1940,NaN,1.4141,-0.5529,-0.6413,...,0,0,0,0,0,0,0,0,1,1
4,方大錚,0,大錚,方,1,1940,NaN,1.4210,-1.0762,-0.3661,...,0,0,0,0,0,0,0,0,1,1


In [82]:
rename_dic = {}
for col in (sampled_df.columns):
    if "FN1" in col:
        rename_dic[col] = col.replace("FN1","FN2")
    elif  "FN2" in col:
        rename_dic[col] = col.replace("FN2","FN1")
rename_dic

{'FN1_wv_0': 'FN2_wv_0',
 'FN2_wv_0': 'FN1_wv_0',
 'FN1_wv_1': 'FN2_wv_1',
 'FN2_wv_1': 'FN1_wv_1',
 'FN1_wv_2': 'FN2_wv_2',
 'FN2_wv_2': 'FN1_wv_2',
 'FN1_wv_3': 'FN2_wv_3',
 'FN2_wv_3': 'FN1_wv_3',
 'FN1_wv_4': 'FN2_wv_4',
 'FN2_wv_4': 'FN1_wv_4',
 'FN1_wv_5': 'FN2_wv_5',
 'FN2_wv_5': 'FN1_wv_5',
 'FN1_wv_6': 'FN2_wv_6',
 'FN2_wv_6': 'FN1_wv_6',
 'FN1_wv_7': 'FN2_wv_7',
 'FN2_wv_7': 'FN1_wv_7',
 'FN1_wv_8': 'FN2_wv_8',
 'FN2_wv_8': 'FN1_wv_8',
 'FN1_wv_9': 'FN2_wv_9',
 'FN2_wv_9': 'FN1_wv_9',
 'FN1_wv_10': 'FN2_wv_10',
 'FN2_wv_10': 'FN1_wv_10',
 'FN1_wv_11': 'FN2_wv_11',
 'FN2_wv_11': 'FN1_wv_11',
 'FN1_wv_12': 'FN2_wv_12',
 'FN2_wv_12': 'FN1_wv_12',
 'FN1_wv_13': 'FN2_wv_13',
 'FN2_wv_13': 'FN1_wv_13',
 'FN1_wv_14': 'FN2_wv_14',
 'FN2_wv_14': 'FN1_wv_14',
 'FN1_wv_15': 'FN2_wv_15',
 'FN2_wv_15': 'FN1_wv_15',
 'FN1_wv_16': 'FN2_wv_16',
 'FN2_wv_16': 'FN1_wv_16',
 'FN1_wv_17': 'FN2_wv_17',
 'FN2_wv_17': 'FN1_wv_17',
 'FN1_wv_18': 'FN2_wv_18',
 'FN2_wv_18': 'FN1_wv_18',
 'FN1_wv_19': 

In [118]:
def FN_augmentation( df ):
    rename_dic = {}
    for col in (df.columns):
        if "FN1" in col:
            rename_dic[col] = col.replace("FN1","FN2")
        elif  "FN2" in col:
            rename_dic[col] = col.replace("FN2","FN1")
    df = pd.concat([df, df.rename(columns = rename_dic)], axis=0)
    
    return df.fillna(0)

(4)Train 多種 gender RFC

In [199]:
sampled_df[ sampled_df.apply(lambda x: x['gender']!=x.mgender,axis=1) ]

,name,BirthYear,FirstName,LastName,gender,message,userID,FN1_wv_0,FN2_wv_0,FN1_wv_1,...,Zodiac_猴,Zodiac_羊,Zodiac_虎,Zodiac_蛇,Zodiac_豬,Zodiac_雞,Zodiac_馬,Zodiac_鼠,Zodiac_龍,mgender
57,何可,0,可,何,1,1940,NaN,0.000000,-2.259699,0.000000,...,0,0,0,0,0,0,0,0,1,0
82,吳國瑜,0,國瑜,吳,1,1940,NaN,3.000507,-1.559357,-0.890834,...,0,0,0,0,0,0,0,0,1,0
121,李湘渝,0,湘渝,李,1,1940,NaN,-1.018383,1.468332,-0.057867,...,0,0,0,0,0,0,0,0,1,0
137,沈春秀,0,春秀,沈,1,1940,NaN,1.752956,-6.028177,-3.599663,...,0,0,0,0,0,0,0,0,1,0
162,林圓,0,圓,林,1,1940,NaN,0.000000,0.936528,0.000000,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138412,郭政豪,12,政豪,郭,0,2002,268428420338176,-1.794844,-3.608984,-3.385827,...,0,0,0,0,0,0,1,0,0,1
138413,黃微明,11,微明,黃,0,1999,913839378766328,-1.063672,-12.395959,-2.606959,...,0,0,0,0,0,0,0,0,0,1
138445,周芷茵,13,芷茵,周,1,2005,1718228048482789,-3.529580,-0.019260,5.995256,...,0,0,0,0,0,1,0,0,0,0
138459,陳以利,10,以利,陳,1,1992,1802015343149713,2.032988,0.312624,-4.790244,...,1,0,0,0,0,0,0,0,0,0


In [60]:
y_feature = 'mgender'

In [119]:
w_io = open("./Training/result_all.csv", "w")
w_io.write("Type,feature,lens,accuracy,F1,Precision,Recall\n")

result_io = open("./Training/result.csv", "w")
result_io.write("feature,lens,accuracy,F1,Precision,Recall\n")

Accuracy_list = []
precision_list = []
recall_list = []
F1_list = []
for i,feature in enumerate(feature_combinations_gender):
    if type(feature)==str:
    #if type(feature)==str and 'w2v' == feature:
        x_feature = get_x_feature ([feature] , Name_df.columns)
    else:
        #if 'w2v' not in  feature:
        #    continue
        x_feature = get_x_feature (feature , Name_df.columns)
    f = ''.join( [ x[0].upper() for x in feature] ).upper()

    print("Combination",i,"Training feature category:",feature)
    print("len of x_feature:",len(x_feature))
    tmp1,tmp2,tmp3,tmp4 = [] , [] , [] , []
    for test_time in range(1):
        
        dev_df = sampled_df.sample( n= len(Name_df)//10, frac=None , replace=False, weights=None, random_state=None, axis=0)
        #train_x, test_x, train_y, test_y = split_dataset( sampled_df.drop(dev_df.index), 0.7, x_feature, y_feature)
        train_x, test_x, train_y, test_y = split_dataset( FN_augmentation(sampled_df.drop(dev_df.index)), 0.7, x_feature, y_feature)
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y.values.reshape(-1, 1).ravel(), estimators_num = 64 , min_samples_leaf_num = 1)
        print('Finished training')
        predictions = trained_model.predict(test_x)
        print('Finished prdeiction')
        
        print("Train Metrics")
        print("train_x len ",len(train_x))
        Accuracy, precision , recall , F1 = RFC_metrics(train_x , train_y  , trained_model )
        w_io.write("Train,"+f+",{},{},{},{},{}\n".format(len(x_feature), Accuracy, precision , recall , F1))
        print("\nTest Metrics")
        print("Test len ",len(test_x))
        Accuracy, precision , recall , F1 = RFC_metrics(test_x , test_y  , trained_model )
        w_io.write("Test,"+f+",{},{},{},{},{}\n".format(len(x_feature),Accuracy, precision , recall , F1))
        print("\nDevelopment Metrics")
        print("nDevelopment len ",len(dev_df))
        Accuracy, precision , recall , F1 = RFC_metrics(dev_df[x_feature] , dev_df[y_feature]  , trained_model )
        w_io.write("Develop,"+f+",{},{},{},{},{}\n".format(len(x_feature),Accuracy, precision , recall , F1))
        tmp1+=[Accuracy]
        tmp2+=[precision]
        tmp3+=[recall]
        tmp4+=[F1]
    avg_Accuracy = round( np.array(tmp1).mean() , 4)
    avg_precision = round( np.array(tmp2).mean() ,4)
    avg_recall = round( np.array(tmp3).mean(),4)
    avg_F1 = round( np.array(tmp4).mean(),4)
    Accuracy_list+=[avg_Accuracy]
    precision_list+=[avg_precision]
    recall_list+=[avg_recall]
    F1_list+=[avg_F1]
    result_io.write(f+",{},{},{},{},{}\n".format(len(x_feature),avg_Accuracy, avg_precision , avg_recall , avg_F1))
    FileName = f + "_gnder_RFC_model.pkl"
    '''
    with open('./TrainedModel/'+ FileName , 'wb') as handle:
          pickle.dump(trained_model, handle)
          print("\Output model Done.")
    '''
    
w_io.close()
result_io.close()

Combination 0 Training feature category: w2v
len of x_feature: 200
estimators_num =  64 min_samples_leaf_num =  1 Training Data len =  174511
Finished training
Finished prdeiction
Train Metrics
train_x len  174511
Accuracy ::  0.9996
Precision_score ::  0.9997
Recall_score ::  0.9997
F1_score ::  0.9997

Test Metrics
Test len  74791
Accuracy ::  0.9428
Precision_score ::  0.9416
Recall_score ::  0.9627
F1_score ::  0.952

Development Metrics
nDevelopment len  13850
Accuracy ::  0.9464
Precision_score ::  0.9431
Recall_score ::  0.9672
F1_score ::  0.955
Combination 1 Training feature category: phonetic
len of x_feature: 321
estimators_num =  64 min_samples_leaf_num =  1 Training Data len =  174511
Finished training
Finished prdeiction
Train Metrics
train_x len  174511
Accuracy ::  0.9283
Precision_score ::  0.9391
Recall_score ::  0.9392
F1_score ::  0.9391

Test Metrics
Test len  74791
Accuracy ::  0.8751
Precision_score ::  0.8908
Recall_score ::  0.8982
F1_score ::  0.8945

Developm

In [66]:
dev_df

,name,BirthYear,FirstName,LastName,gender,message,userID,FN1_wv_0,FN2_wv_0,FN1_wv_1,...,Zodiac_狗,Zodiac_猴,Zodiac_羊,Zodiac_虎,Zodiac_蛇,Zodiac_豬,Zodiac_雞,Zodiac_馬,Zodiac_鼠,Zodiac_龍
121643,郭秀萍,6,秀萍,郭,0,1971,1096256097143074,-6.028177,-1.669684,-5.868910,...,0,0,0,0,0,1,0,0,0,0
72053,陳雪霞,2,雪霞,陳,0,1954,NaN,1.360215,1.348958,4.014402,...,0,0,0,0,0,0,0,1,0,0
66230,葉競棠,2,競棠,葉,0,1951,NaN,-1.867900,-1.954202,-5.025996,...,0,0,0,0,0,0,0,0,0,0
61165,于曉雯,4,曉雯,于,0,1960,NaN,-4.979229,-1.476311,1.824066,...,0,0,0,0,0,0,0,0,1,0
12409,徐質敏,2,質敏,徐,1,1951,NaN,-1.830149,-6.833077,-4.545825,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7917,呂聯東,1,聯東,呂,1,1949,NaN,3.828465,5.197540,-15.155359,...,0,0,0,0,0,0,0,0,0,0
113859,高小妹,6,小妹,高,0,1973,215758365587490,-2.174975,3.197350,4.283507,...,0,0,0,0,0,0,0,0,0,0
128430,朱睿紳,9,睿紳,朱,1,1988,1635708209780201,-2.356484,0.194362,1.819373,...,0,0,0,0,0,0,0,0,0,1
18746,李建棠,2,建棠,李,1,1954,NaN,8.122230,-1.954202,-3.091460,...,0,0,0,0,0,0,0,1,0,0


In [69]:
train_x, test_x, train_y, test_y = split_dataset( sampled_df.drop(dev_df.index), 0.7, x_feature, y_feature)

(5)save Model

In [122]:
feature =['w2v']
x_feature = get_x_feature (feature , Name_df.columns)

In [131]:

train_x, test_x, train_y, test_y = split_dataset( FN_augmentation(sampled_df.drop(dev_df.index)), 0.7, x_feature, y_feature)

trained_model = random_forest_classifier(train_x, train_y.values.reshape(-1, 1).ravel(), estimators_num = 64 , min_samples_leaf_num = 1)
print('Finished training')
predictions = trained_model.predict(test_x)
print('Finished prdeiction')

print("Train Metrics")
print("train_x len ",len(train_x))
Accuracy, precision , recall , F1 = RFC_metrics(train_x , train_y  , trained_model )
print("\nTest Metrics")
print("Test len ",len(test_x))
Accuracy, precision , recall , F1 = RFC_metrics(test_x , test_y  , trained_model )

f = ''.join( [ x[0].upper() for x in feature] ).upper()
FileName = f + "_gnder_RFC_model.pkl"

with open('./TrainedModel/'+ FileName , 'wb') as handle:
    pickle.dump(trained_model, handle)
    print("Output model Done.")
    
with open('./TrainedModel/'+ f +"_feature_list.pkl" , 'wb') as handle:
    pickle.dump(x_feature, handle)
    print("Output feature Done.")


estimators_num =  64 min_samples_leaf_num =  1 Training Data len =  174511
Finished training
Finished prdeiction
Train Metrics
train_x len  174511
Accuracy ::  0.9973
Precision_score ::  0.9985
Recall_score ::  0.9987
F1_score ::  0.9986

Test Metrics
Test len  74791
Accuracy ::  0.9902
Precision_score ::  0.9921
Recall_score ::  0.9977
F1_score ::  0.9949
Output model Done.
Output feature Done.


(3-5) metric評量

In [62]:
def RFC_metrics(  data_x , data_y , trained_model ):
        cf_matrix = confusion_matrix(data_y, trained_model.predict(data_x))
        TN = cf_matrix[0][0]
        FP = cf_matrix[0][1]
        FN = cf_matrix[1][0]
        TP = cf_matrix[1][1]
        precison = TP / (TP + FP)
        recall = TP / (TP + FN)
        accuracy = (TP + TN) / (TN + FP + FN + TP)
        F1 = 2*precison*recall / (precison + recall)
        print("Accuracy :: ", round( accuracy, 4) )
        print("Precision_score :: ", round( precison,4)) 
        print("Recall_score :: ", round(recall,4))
        print("F1_score :: ", round(F1,4) )
        return round( accuracy, 4) , round( precison,4) , round(recall,4) ,  round(F1,4)



In [97]:
target_names = [ '女性' ,'男性']

print("report:\n",classification_report(test_y, predictions ,target_names= target_names))

report:
               precision    recall  f1-score   support

          女性       0.88      0.84      0.86     15534
          男性       0.89      0.91      0.90     21862

    accuracy                           0.88     37396
   macro avg       0.88      0.88      0.88     37396
weighted avg       0.88      0.88      0.88     37396



In [ ]:
import datetime
# multi-answer RFC
def basic_RFClassifier_ts(data_df,x_feature,y_feature,Validation_times,mode,train_index,test_index,random_avg_year_error):
    single_year_number=0
    print(datetime.datetime.now())
    #N fold cross-validation
    Train_accuracy_score=0
    Test_accuracy_score=0
    
    avg_dist_error = 0
    avg_random_score=0
    avg_random_dist_error=0
    avg_single_year_errorr=0
    avg_year_error = [0 for x in range(10) ]
    if mode!=2:
        random_avg_year_error = [0 for x in range(10) ]
    for i in range(Validation_times):
        random_score=0
        if mode==0:
            train_x, test_x, train_y, test_y = split_dataset(data_df, 0.7, x_feature, y_feature)
            train_index = train_x.index
            test_index = test_x.index
        else:
            train_x = data_df.loc[train_index][x_feature]
            test_x =data_df.loc[test_index][x_feature]
            train_y = data_df.loc[train_index][y_feature]
            train_y = train_y.values.reshape(-1, 1)
            test_y = data_df.loc[test_index][y_feature]
            test_y = test_y.values.reshape(-1, 1)
        
        # Create random forest classifier instance
        trained_model = random_forest_classifier(train_x, train_y.ravel())
        print('Finished training')
        
        predictions = trained_model.predict(test_x)
        print('Finished prdeiction')
        # Train and Test Accuracy
        if(Validation_times!=1):
            print('Run times:',i+1)

        test_name_dict_list= get_test_name_dict_list(test_index,data_df,'BirthYear')
        test_name_year_dict_list =  get_test_name_dict_list(test_index,data_df,'message')
        print('test dataset 有',len(test_x),'個名字,',len(test_name_dict_list),'種名字')    
        
        test_name_list = data_df.loc[test_index].FirstName.tolist()
        
        dist_error=0
        accuracy=0
        random_dist_error=0
        single_year_error=0
        year_error = [0 for x in range(10) ]
        year_count = [0 for x in range(10) ]
        
        if mode!=2:
            random_year_error = [0 for x in range(10) ]
            
        for test_no,year in enumerate(predictions):
            

            #print(name)
            name = test_name_list[test_no]
            if name in test_name_dict_list:
                year_count[ len(test_name_dict_list[name])-1 ]+=1

                if year in test_name_dict_list[name]:
                    accuracy+=1
                else:
                    #計算錯誤，把平均絕對值誤差加上去
                    error = get_min_distance(year,test_name_year_dict_list[name]) 
                    dist_error += error
                    
                    year_error[ len(test_name_dict_list[name])-1 ]+=error
      
                if mode!=2:
                    random_year = random.randint(1,10)
                    if random_year in test_name_dict_list[name]:
                        random_score+=1
                    else:
                        error = get_min_distance(random_year,test_name_year_dict_list[name])
                        random_dist_error += error
                        random_year_error[ len(test_name_dict_list[name])-1 ] += error
            else:
                print('wtf test name not in test_name_dict_list')
        
        accuracy/=len(predictions)      

        print ("Train Accuracy :: ", accuracy_score(train_y, trained_model.predict(train_x)))
        print ("Test Accuracy  :: ", accuracy)
        print('平均年份絕對值誤差::',dist_error/len(predictions) )
        #print('只在某一年份出現的平均年份絕對值誤差::',single_year_error/single_year_number)
            
        for i in range(len(year_error)):
            avg_year_error[i]+=year_error[i]/year_count[i]
            if mode!=2:
                random_avg_year_error[i] = random_year_error[i]/year_count[i]
                random_avg_year_error[i]+=random_year_error[i]/year_count[i]
            print('名字只出現在共',i+1,'個年份區間的名字比例::',round(year_count[i]/len(predictions),4)*100,'%')
            print('名字只出現在共',i+1,'個年份區間的平均誤差::',round(year_error[i]/year_count[i],4))
            if mode!=2:
                print('random 名字只出現在共',i+1,'個年份區間的平均誤差::',round(random_year_error[i]/year_count[i],4))
        if mode==1:    
            print('Random guess accuracy::',random_score/len(predictions))
            print('Random guess error::',random_dist_error/len(predictions))

        #print (" Confusion matrix\n ", confusion_matrix(test_y, predictions))
        
        Train_accuracy_score+=accuracy_score(train_y, trained_model.predict(train_x))
        Test_accuracy_score+= accuracy
        avg_dist_error+=dist_error/len(predictions)
        avg_random_score+=random_score/len(predictions)
        avg_random_dist_error=random_dist_error/len(predictions)
        
#         calculate_errorr_multi_answer( predictions, test_x ,test_name_dict_list,data_df)
#         validate_error_multi_answer( predictions, data_df,test_y,test_x ,test_name_dict_list)
        
    if(Validation_times!=1):
        Train_accuracy_score/=Validation_times
        Test_accuracy_score/=Validation_times
        print ("Average Train Accuracy :: ", Train_accuracy_score)
        print ("Average Test Accuracy  :: ",Test_accuracy_score)
        print('Average 平均年份絕對值誤差::',avg_dist_error/Validation_times )
        for i in range(len(year_error)):
            print('Average 出現在共',i+1,'年份區間的平均誤差::',round(avg_year_error[i]/Validation_times,4))
            print('random 名字只出現在共',i+1,'個年份區間的平均誤差::',round(random_avg_year_error[i]/Validation_times,4))
        if mode!=2:
            print('Average Random guess accuracy::',avg_random_score/Validation_times)
            print('Average Random guess error::',avg_random_dist_error/Validation_times)  
        
        for i in range(9):
            improve_rate = abs(avg_year_error[i]-random_avg_year_error[i])/random_avg_year_error[i]
            print('名字只出現在共',i+1,'個年份區間的的improve rate:',round(improve_rate*100,2),'%')
    return random_avg_year_error